In [3]:
import os 


In [6]:
%pwd

'd:\\cancer-classification-using-MLflow'

In [5]:
os.chdir("../")

In [31]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    trained_model_path:Path
    updated_base_model_path: Path 
    training_data : Path
    validation_data : Path
    params_epochs : int 
    params_batch_size : int
    params_is_augmentation : bool
    params_image_size : list
    

In [32]:
from cnnclassifier.constants import *
from cnnclassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [33]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir , "data","train")
        validation_data = os.path.join(self.config.data_ingestion.unzip_dir , "data","valid")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir= Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            validation_data= Path(validation_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )

        return training_config

    

In [34]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from tensorflow.keras.applications.resnet import preprocess_input



In [45]:
class Training:
    def __init__(self,config:TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
        
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs,
            dtype= 'float32',
            preprocessing_function= preprocess_input
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.validation_data,
            class_mode= "categorical",
            **dataflow_kwargs
            )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                preprocessing_function = preprocess_input,
                rotation_range = 10,
                width_shift_range = 0.3,
                height_shift_range = 0.3,
                shear_range = 0.2,
                zoom_range = 0.1,
                horizontal_flip = True,
                vertical_flip = True,
                dtype = 'float32'
                )
        else:
            train_datagenerator = valid_datageneratior

        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            class_mode= "categorical",
            **dataflow_kwargs
        )

    @staticmethod 
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)
    
    def train(self):

        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
            )
        
        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )




In [46]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e



[2024-06-09 22:44:31,428 : INFO: common: yaml file:config\config.yaml loaded successfully]
[2024-06-09 22:44:31,430 : INFO: common: yaml file:params.yaml loaded successfully]
[2024-06-09 22:44:31,431 : INFO: common: created directory at: artifacts]
[2024-06-09 22:44:31,432 : INFO: common: created directory at: artifacts\training]
Found 72 images belonging to 4 classes.
Found 613 images belonging to 4 classes.
38/38 [==============================] - 127s 3s/step - loss: 10.4931 - accuracy: 0.3266 - val_loss: 1.5465 - val_accuracy: 0.1719
